In [ ]:
import netsquid as ns
# Set qstate formalism to use density matrices.
ns.set_qstate_formalism(ns.QFormalism.DM)

# Exercise solution


## Network setup

In [ ]:
from netsquid.nodes import Node
from netsquid.components import Port, QuantumChannel
from netsquid.nodes import DirectConnection
from netsquid.components.models import FixedDelayModel, DepolarNoiseModel, DephaseNoiseModel

# Create Node objects
alice_node = Node("Alice")
bob_node   = Node("Bob")

# Create Port objects
alice_port = Port("Alice_port", alice_node)
bob_port   = Port("Bob_port",   bob_node)

# Create delay and noise models
delay_model = FixedDelayModel(delay=10.)
qerror_model = DephaseNoiseModel(dephase_rate=0.20, time_independent=True)

# Create channels and connection between Alice and Bob
channel_a_to_b = QuantumChannel("Channel Alice to Bob",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": qerror_model}
                                )

channel_b_to_a = QuantumChannel("Channel Bob to Alice",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": qerror_model}
                                  )

connection = DirectConnection(name="connection",
                              channel_AtoB=channel_a_to_b,
                              channel_BtoA=channel_b_to_a)


# Connect Alice with Bob using the newly created connection and ports
alice_node.connect_to(remote_node=bob_node, connection=connection,
                     local_port_name="Alice_port", remote_port_name="Bob_port")

## Protocols

In [ ]:
from netsquid.protocols import NodeProtocol

# Create protocol classes for Alice and Bob nodes
class AliceProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Alice_port"]

        # Create qubits q1 and q2 in the ground state
        q1, q2 = ns.qubits.create_qubits(2)

        # Entangle q1 and q2 to the Phi+ bell state 
        ns.qubits.operate(q1, ns.H)
        ns.qubits.operate([q1, q2], ns.CX)

        # Send qubit q2 to Bob
        port.tx_output(q2)
        print(f"{ns.sim_time()} ns Alice send q2")

        # Wait 30 ns
        yield self.await_timer(30)

        # Measure qubit q1
        m, _ = ns.qubits.measure(q1)
        print(f"{ns.sim_time()} ns Alice measures q1: {m}")


class BobProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Bob_port"]

        # Wait for the message with qubit, q2, from Alice to arrive
        yield self.await_port_input(port)
        q2 = port.rx_input().items[0]

        # Obtain full density matrix
        # q2.qstate.qubits will return a list of all qubits that are part of the quantum state
        dm = ns.qubits.reduced_dm(q2.qstate.qubits)
        print(f"{ns.sim_time()} ns Bob receives q2, DM=\n{dm}")

        # Measure qubit q2
        m, _ = ns.qubits.measure(q2)
        print(f"{ns.sim_time()} ns Bob measures q2: {m}")

# Create protocol instances with reference to the node the protocol will run on
alice_protocol = AliceProtocol(node=alice_node)
bob_protocol = BobProtocol(node=bob_node)

## Run simulation

In [ ]:
# Start all protocols
alice_protocol.start()
bob_protocol.start()

# Run simulation
sim_stats = ns.sim_run()

# Show simulation statistics
print(sim_stats)

# Reset simulation
alice_protocol.stop()
bob_protocol.stop()

# Reset the simulation: clear scheduled events, entities and event handlers & Reset simulation time to 0
ns.sim_reset()

## Optional exercise solution

## Protocols

In [ ]:
from netsquid.protocols import NodeProtocol

# Create protocol classes for Alice and Bob nodes
class AliceProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Alice_port"]

        # Create qubits q1 and q2 in the ground state
        q1, q2 = ns.qubits.create_qubits(2)

        # Entangle q1 and q2 to the Phi+ bell state 
        ns.qubits.operate(q1, ns.H)
        ns.qubits.operate([q1, q2], ns.CX)

        # Send qubit q2 to Bob
        port.tx_output(q2)
        print(f"{ns.sim_time()} ns Alice send q2")

        # Wait for the message with returned qubit, q2, from Bob to arrive
        yield self.await_port_input(port)
        q2 = port.rx_input().items[0]
        print(f"{ns.sim_time()} ns Alice received q2, \nDM:\n{ns.qubits.reduced_dm([q1, q2])}")

        # measure qubits q1 and q2
        m1, _ = ns.qubits.measure(q1)
        m2, _ = ns.qubits.measure(q2)
        print(f"{ns.sim_time()} ns Alice measures q1: {m1} and q2: {m2}")

        # Check if measurements are equal to recover the secret bit value
        if m1 == m2:
            print("Alice concludes secret bit is 0")
        else:
            print("Alice concludes secret bit is 1")



class BobProtocol(NodeProtocol):
    # override __init__ to add ability to set the secret bit as an instance attribute
    def __init__(self, node, secret_bit: bool):
        super().__init__(node)
        self.secret_bit = secret_bit
    
    def run(self):
        port = self.node.ports["Bob_port"]

        # Wait for the message with qubit, q2, from Alice to arrive
        yield self.await_port_input(port)
        q2 = port.rx_input().items[0]
        print(f"{ns.sim_time()} ns Bob receives q2")

        # Conditional on secret bit value, apply the X gate on q2
        if self.secret_bit == 1:
            ns.qubits.operate(q2, ns.X)

        # Send qubit q2 back to Alice
        port.tx_output(q2)


## Run simulation

In [ ]:
# Reset simulation
ns.sim_reset()

# Choose secret bit value
secret_bit = 1

# Create protocol instances
alice_protocol = AliceProtocol(node=alice_node)
bob_protocol = BobProtocol(node=bob_node, secret_bit=secret_bit)

# Start all protocols
alice_protocol.start()
bob_protocol.start()

# Run simulation
sim_stats = ns.sim_run()

# Show simulation statistics
print(sim_stats)

# Stop protocols
alice_protocol.stop()
bob_protocol.stop()